# Data Modelling

In [83]:
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from helpers.helper_functions import translate_to_file_string
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics

inputFile = translate_to_file_string("./data/Data_Preparation_Result.csv")

## Create Spark Session

In [84]:
#create a SparkSession
spark = (SparkSession
       .builder
       .appName("DataModelling")
       .getOrCreate())
# create a DataFrame using an ifered Schema 
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ";") \
       .csv(inputFile)   
print(df.printSchema())

root
 |-- Bundesland: string (nullable = true)
 |-- BundeslandIndex: integer (nullable = true)
 |-- Landkreis: string (nullable = true)
 |-- LandkreisIndex: integer (nullable = true)
 |-- Altersgruppe: string (nullable = true)
 |-- AltersgruppeIndex: double (nullable = true)
 |-- Geschlecht: string (nullable = true)
 |-- GeschlechtIndex: double (nullable = true)
 |-- FallStatus: string (nullable = true)
 |-- FallStatusIndex: double (nullable = true)
 |-- Falldatum: string (nullable = true)

None


## Vorbereitung der Daten

### Filtern der Datensätze
Für das Training dieses Modells ist es sinnvoll nur die Fälle zu betrachten, bei den der Ausgang der Corona-Erkrankung bereits bekannt ist ("GENESEN" oder "GESTORBEN"). Daher werden die Fälle mit noch erkrankten Personen herausgefiltert. Ebenfalls muss der FallStatusIndex neu vergeben werden, damit dieses Feature nur noch die Werte 0 oder 1 enthält.

In [85]:
dfNeu = df.filter(df.FallStatus != "NICHTEINGETRETEN").drop("FallStatusIndex")

### FallStatusIndex

In [86]:
# Wollten eigentlich den Indexer mitgeben in die Pipeline. Dies führt aber zum Fehler, dass er das Label nicht kennt.
indexer = StringIndexer(inputCol="FallStatus", outputCol="FallStatusIndex")
dfReindexed = indexer.fit(dfNeu).transform(dfNeu)

### Ziehen eines Samples
Da der Datensatz sehr groß ist, kommt es tlw. während dem Berechnen der Statistiken mittels der Multiclass-Metrics zu Problemen. Die Statistik enthält dann nur Nullwerte. Daher wird vorher ein Sample aus dem Datensatz gezogen.

In [87]:
dfsample = dfReindexed.sample(withReplacement=False, fraction=0.01, seed=12334556)

### Undersampling
Ähnlich dem Fraud-Detection-Beispiel von Tara Boyle (2019) ist die Klasse der an Corona-Verstorbenen im vorliegenden Datensatz unterrepresentiert, weshalb man an dieser Stelle von einer Data Imbalance spricht. Dies sieht man wenn man die Anzahl der Todesfälle mit den Anzahl der Genesenen vergleicht.

In [88]:
# Vergleich der Fallzahlen
dfsample.groupBy("FallStatus").count().show()

+----------+-----+
|FallStatus|count|
+----------+-----+
|   GENESEN|34657|
| GESTORBEN|  950|
+----------+-----+



Die meisten Machine Learning Algorithmen arbeiten am Besten wenn die Nummer der Samples in allen Klassen ungefähr die selbe größe haben. Dies konnte auch im Zuge dieser Arbeit bei den unterschiedlichen Regressions-Modellen festgestellt werden. Da die einzelnen Modelle versuchen den Fehler zu reduzieren, haben alle Modelle am Ende für einen Datensatz nur die Klasse Genesen geliefert, da hier die Wahrscheinlichkeit am größten war korrekt zu liegen. 
Um diese Problem zu lösen gibt es zwei Möglichkeiten: Under- und Oversampling. Beides fällt unter den Begriff Resampling
Beim Undersampling werden aus der Klasse mit den meißten Instanzen, Datensätze gelöscht, wohingegen beim Oversampling, der Klasse mit den wenigsten Isntanzen, neue Werte hinzugefügt werden. (Will Badr 2019; Tara Boyle 2019)
Da in diesem Fall ausreichend Datensätze vorhanden sind, bietet sich Ersteres an.

In [89]:
# Ermittlung der Anzahl dr Verstorbenen
dfGestorben = dfsample.filter(dfsample.FallStatus == "GESTORBEN")
anzahlGestorben = dfGestorben.count()
print("Anzahl Gestorben : %s" % anzahlGestorben)

Anzahl Gestorben : 950


In [90]:
# Ermittlung des Verhätlnisses von Verstorben und Gensen
dfGenesen = dfsample.filter(dfsample.FallStatus == "GENESEN")
anzahlGenesen = dfGenesen.count()
print("Anzahl Genesen : %s" % anzahlGenesen)

ratio = anzahlGestorben / anzahlGenesen
print("Verhältnis : %s" % ratio)



Anzahl Genesen : 34657
Verhältnis : 0.027411489742331995


In [91]:
# Ziehen eines Samples mit der näherungsweise selben Anzahl wie Verstorbene
dfGenesenSample = dfGenesen.sample(fraction=ratio, seed=12345)

In [92]:
dfGesamtSample = dfGestorben.union(dfGenesenSample)
# Kontrolle
dfGesamtSample.groupBy("FallStatus").count().show()

+----------+-----+
|FallStatus|count|
+----------+-----+
|   GENESEN|  926|
| GESTORBEN|  950|
+----------+-----+



### Splitten in Trainings und Testdaten

In [93]:
splits = dfGesamtSample.randomSplit([0.8, 0.2], 345678)
trainingData = splits[0]
testData = splits[1]

### Aufbau des Feature-Vectors

In [94]:
assembler =  VectorAssembler(outputCol="features", inputCols=["GeschlechtIndex","AltersgruppeIndex", "LandkreisIndex","BundeslandIndex"])

## Modellierung
### Linear Regression

In [95]:
lr = LinearRegression(maxIter=10, featuresCol="features", labelCol="FallStatusIndex")

### Pipeline

In [96]:
pipeline = Pipeline(stages=[assembler,lr])

### Evaluator
Für die spätere Cross-Validaton wird ein Evaluator benötigt. Letzterer ist zu wählen, abhängig von dem jeweilligen Modell und Anwendungsfall. (Apache Spark 2020a) In diesem Fall wird ein BinaryClassificationEvaluator angewendet. Dieser eignet sich besonders für binäre Werte. (Apache Spark 2021a) Da Geschlecht, in diesem Fall, der FallStatus 0 oder 1 annehmen kann, bietet er sich hier besonders an.

In [97]:
# Definition des Evaluators
evaluator= BinaryClassificationEvaluator(labelCol="FallStatusIndex",rawPredictionCol="prediction", metricName="areaUnderPR")

### Parametertuning
Eine wichtige Aufgabe beim Machine Learning ist die Auswahl des geeigneten Modells bzw. die passenden Paramter für ein Modell herauszufinden. Letzteres wird auch Parametertuning genannt. Die in Pyspark enthaltene MLLib bietet speziell hierfür ein entsprechende Tooling. Und zwar kann ein CrossValidator bzw. ein TrainValidationSplit verwendet werden. Voraussetzung sind ein Estimator (ein Modell oder eine Pipeline), ein Paramter-Grid und eine Evaluator. Dies ist auch im Zusammenhang mit dem Thema Cross-Validation zu sehen. (Apache Spark 2020a)

In [98]:
paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.3,0.01]) \
    .addGrid(lr.elasticNetParam, [0.8])\
    .addGrid(evaluator.metricName, ["areaUnderPR", "areaUnderROC"])\
    .build()

### Cross-Validation

In [99]:
# Definition des Cross-Validators 
# num-Folds gibt an in wie viele Datensatz-Paare die Datensätze aufgeteilt werden.
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2,
                          parallelism=2)

#### Durchführung

In [100]:
# Anpassung des Modells und Auswahl der besten Parameter
cvModel = crossval.fit(trainingData)

### Testen des Modells

In [101]:
predictions = cvModel.transform(testData)
predictions.show()

+--------------------+---------------+--------------------+--------------+------------+-----------------+----------+---------------+----------+----------+---------------+--------------------+-------------------+
|          Bundesland|BundeslandIndex|           Landkreis|LandkreisIndex|Altersgruppe|AltersgruppeIndex|Geschlecht|GeschlechtIndex|FallStatus| Falldatum|FallStatusIndex|            features|         prediction|
+--------------------+---------------+--------------------+--------------+------------+-----------------+----------+---------------+----------+----------+---------------+--------------------+-------------------+
|   Baden-Württemberg|              8|         SK Mannheim|          8222|     A60-A79|              2.0|         W|            0.0| GESTORBEN|2021-01-10|            1.0|[0.0,2.0,8222.0,8.0]|0.48967113536628104|
|              Bayern|              9|           LK Dachau|          9174|        A80+|              3.0|         W|            0.0| GESTORBEN|2021-03-2

In [102]:
# Kontrolle der Predictions
predictions.groupBy("FallStatusIndex", "prediction").count().show()

+---------------+-------------------+-----+
|FallStatusIndex|         prediction|count|
+---------------+-------------------+-----+
|            0.0| 0.5005728352873624|    1|
|            1.0| 0.5016809758688531|    1|
|            0.0| 1.0270812266630422|    1|
|            0.0| 0.4768734876328377|    1|
|            1.0| 0.7437353142384573|    1|
|            1.0| 0.7032721076861843|    1|
|            1.0| 0.5174074385064675|    1|
|            1.0|0.49353830448068653|    1|
|            1.0| 0.7206964340363617|    1|
|            0.0| 0.1365031230263552|    1|
|            0.0|0.13026800305101113|    1|
|            0.0|0.17212741799782375|    1|
|            0.0|0.15202188186822405|    1|
|            1.0| 0.7232477587735309|    1|
|            1.0| 0.6827633289248509|    2|
|            1.0| 0.6254109076009013|    1|
|            1.0| 0.7246697196328022|    1|
|            1.0| 0.6842701829697506|    1|
|            0.0|  0.863355000899676|    1|
|            0.0|0.2680689803009

## Modell - Evaluation

Area Under PR

In [103]:
accuracy = evaluator.evaluate(predictions)
print("Test Error",(1.0 - accuracy))

Test Error 0.29971359540241593


### BinaryClassificationMetrics

Bei dem untersuchten Label (FallStatus mit den Ausprägungen Verstorben und Genesen) handelt es sich um ein einen BinaryClasificator. Er kann die Werte 0 und 1 annehmen. Für die Modellevaluation sind daher die BinaryClassificationMetrics zu verwenden. (Apache Spark 2021i)

In [104]:
predictionAndLabels = predictions.select("prediction", "FallStatusIndex").rdd.map(lambda p: [p[0], p[1]]) # Map to RDD prediction|label

In [105]:
# Instanzieire das BinaryClassificationMetrics-Objekt
metrics = BinaryClassificationMetrics(predictionAndLabels)

# Fläche unter der Precision-recall Curve

print("Area under PR = %s" % metrics.areaUnderPR)

# Fläche unter der ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)

Area under PR = 0.7002864045975841
Area under ROC = 0.8237604540023895


### Multiclass classification Metrics
In den meißten Fällen können auch Multiclass Classification Metrics bei Binary Classifaction Problemen angewandt werden.

In [106]:
predictionAndLabels = predictions.select("prediction", "FallStatusIndex").rdd.map(lambda p: (p[0], p[1])) # Map to RDD prediction|label
# Instantiate metrics object
mcMetrics = MulticlassMetrics(predictionAndLabels)



In [107]:
mcMetrics.confusionMatrix().toArray()

array([[0., 0.],
       [0., 0.]])

In [108]:
# Overall statistics
precision = mcMetrics.precision(1.0)
recall = mcMetrics.recall(1.0)
f1Score = mcMetrics.fMeasure(1.0)
print("Summary Stats")
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1 Score = %s" % f1Score)


labels = predictions.select("FallStatusIndex").rdd.map(lambda lp: lp.FallStatusIndex).distinct().collect()
for label in sorted(labels):
    print("Class %s precision = %s" % (label, mcMetrics.precision(label)))
    print("Class %s recall = %s" % (label, mcMetrics.recall(label)))
    print("Class %s F1 Measure = %s" % (label, mcMetrics.fMeasure(label, beta=1.0)))

# Weighted stats
print("Weighted recall = %s" % mcMetrics.weightedRecall)
print("Weighted precision = %s" % mcMetrics.weightedPrecision)
print("Weighted F(1) Score = %s" % mcMetrics.weightedFMeasure())
print("Weighted F(0.5) Score = %s" % mcMetrics.weightedFMeasure(beta=0.5))
print("Weighted false positive rate = %s" % mcMetrics.weightedFalsePositiveRate)

Summary Stats
Precision = 0.0
Recall = 0.0
F1 Score = 0.0
Class 0.0 precision = 0.0
Class 0.0 recall = 0.0
Class 0.0 F1 Measure = 0.0
Class 1.0 precision = 0.0
Class 1.0 recall = 0.0
Class 1.0 F1 Measure = 0.0
Weighted recall = 0.0
Weighted precision = 0.0
Weighted F(1) Score = 0.0
Weighted F(0.5) Score = 0.0
Weighted false positive rate = 0.0
